# Packages

In [1]:
# Packages used for the project
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
import requests
#from PIL import Image
#from io import BytesIO

# Gather the Data

## Enhanced Twitter Archive 

We have this file on hand and so have downloaded it manually

In [2]:
# twitter archive is downloaded manually as a file we have on hand
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

## Image Predictions

The file (image_predictions.tsv) hosted on Udacity's servers and was downloaded programmatically using the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [3]:
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
# Download the image_predictions.tsv programmatically from Udacity's servers
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

<Response [200]>

In [4]:
with open(os.path.join(folder_name, 
                      url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [6]:
image_predictions = pd.read_csv('U:/Professional Development/Udacity/Projects/T2 P2/image_predictions/image-predictions.tsv', sep='\t')

## Twitter API

Tweet ID, retweet count, favorite count and date created queried from Twitter's API

In [ ]:
# Create authentication to Twtitter's API using Python's Tweepy library
# Take out consumer key, consumer secret, access token and access secret before submitting the project
consumer_key = 'consumer_key'
consumer_secret = 'consumer_secret'
access_token = 'access_token'
access_secret = 'access_secret'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [ ]:
#Compiling the list of tweet ids from the twitter_archive
tweet_id_list = twitter_archive['tweet_id']
tweet_id_list

In [ ]:
#querying twitter's api to get the retweet count, favorite count and the date created.
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

#reply_count: Number of times this Tweet has been replied to
#retweet_count: Number of times this Tweet has been retweeted
#favorite_count: Indicates approximately how many times this Tweet has been liked by Twitter users.
#created_at: UTC time when this Tweet was created
#withheld_in_countries: When present, indicates a list of uppercase two-letter country codes this content is withheld from. Twitter supports the following non-country values for this field:
#“XX” - Content is withheld in all countries “XY” - Content is withheld due to a DMCA request.

tweet_count = []
tweet_errors = []
for tweetid in tweet_id_list:
    try:
        tweet = api.get_status(tweetid, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        print(tweetid)
        retweet_count = tweet.retweet_count
        favorite_count = tweet.favorite_count
        created_at = tweet.created_at
        tweet_count.append({'tweet_id': str(tweetid),
                            'retweet_count': retweet_count,
                            'favorite_count': favorite_count,
                            'created_at': str(created_at)})
    except Exception as e:
        print(str(tweetid) + str(e))
        tweet_errors.append({'tweet_id': tweetid})


with open('tweet_json.txt', 'w') as outfile:  
    json.dump(tweet_count, outfile)     

In [7]:
# load the json file
with open('tweet_json.txt') as json_file:  
    tweet_count_list = json.load(json_file)

In [8]:
# Create a pandas data frame from the json file
tweet_count_list = pd.DataFrame(tweet_count_list, columns = ['tweet_id', 'retweet_count', 'favorite_count', 'created_at'])
tweet_count_list = tweet_count_list.sort_values('tweet_id').reset_index(drop=True)
tweet_count_list.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2351 entries, 0 to 2350
Data columns (total 4 columns):
tweet_id          2351 non-null object
retweet_count     2351 non-null int64
favorite_count    2351 non-null int64
created_at        2351 non-null object
dtypes: int64(2), object(2)
memory usage: 73.5+ KB


In [9]:
# Create a pandas data frame for the tweets with a tweet_id error
tweet_errors = pd.DataFrame(tweet_errors, columns = ['tweet_id'])
tweet_errors.to_csv('tweet_errors.csv', index=False)

NameError: name 'tweet_errors' is not defined

# Assessing the Data

In [10]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [12]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [11]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [14]:
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [15]:
tweet_count_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2351 entries, 0 to 2350
Data columns (total 4 columns):
tweet_id          2351 non-null object
retweet_count     2351 non-null int64
favorite_count    2351 non-null int64
created_at        2351 non-null object
dtypes: int64(2), object(2)
memory usage: 73.5+ KB


In [16]:
tweet_count_list.describe()

,retweet_count,favorite_count
count,2351.000000,2351.000000
mean,3127.648660,8106.415993
std,5232.892613,11980.413387
min,0.000000,0.000000
25%,617.000000,1417.500000
50%,1451.000000,3589.000000
75%,3618.500000,10106.000000
max,78939.000000,132040.000000


# Initial Issues

## Quality 

  - We only want original ratings (no retweets) that have images
  - Some dogs don't have a dog type
  - Missing image predictions
  - Missing dog type are not NAN, but 'none'
  - Incorrect ratings example tweet id 83524643952984... should be 13/10 not 960 / 0
  - Incorrect dog names
  - Incomplete dog stages
  - Invalid tweet ids {'tweet_id': 888202515573088257},
 {'tweet_id': 869988702071779329},
 {'tweet_id': 861769973181624320},
 {'tweet_id': 802247111496568832},
 {'tweet_id': 775096608509886464}] 
 
## Tidiness

  - Twitter archive has a column for each dog type
  - Dog ratings and dog type predictions in different files

## Copy

Make a copy of the three datasets to work on

In [48]:
twitter_archive_clean = twitter_archive.copy()
image_predictions_clean = image_predictions.copy()
tweet_count_clean = tweet_count_list.copy()

# Cleaning

## Quality Issue 1

We only want original ratings (no retweets) that have images

## Clean

Remove any tweet information with a retweeted status id

In [49]:
twitter_archive_clean = twitter_archive_clean[pd.isnull(twitter_archive_clean['retweeted_status_id'])]

In [50]:
# Test

twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2175 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2175 non-null object
source                        2175 non-null object
text                          2175 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2117 non-null object
rating_numerator              2175 non-null int64
rating_denominator            2175 non-null int64
name                          2175 non-null object
doggo                         2175 non-null object
floofer                       2175 non-null object
pupper                        2175 non-null object
puppo                         2175 non-null object
dtypes: float64(4), int64(3), object(1

## Quality Issue 2 

Not all dog stages are recorded from the text

## Clean

Extract additional dog stages from the text

In [51]:
twitter_archive_clean['doggo'] = twitter_archive_clean['doggo'].replace('None', twitter_archive_clean.text.str.extract('(dogg\w*)', expand=False))
twitter_archive_clean['floofer'] = twitter_archive_clean['floofer'].replace('None', twitter_archive_clean.text.str.extract('(floof\w*)', expand=False))
twitter_archive_clean['pupper'] = twitter_archive_clean['pupper'].replace('None', twitter_archive_clean.text.str.extract('(pupper\w*)', expand=False))
twitter_archive_clean['puppo'] = twitter_archive_clean['puppo'].replace('None', twitter_archive_clean.text.str.extract('(puppo\w*)', expand=False))


In [52]:
# test
twitter_archive_clean['doggo'].value_counts()

doggo      87
doggos      8
doggles     3
doggy       2
dogg        2
Name: doggo, dtype: int64

In [53]:
twitter_archive_clean['floofer'].value_counts()

floof      15
floofer    10
floofy      3
floofs      1
Name: floofer, dtype: int64

In [54]:
twitter_archive_clean['pupper'].value_counts()

pupper          234
puppers          23
puppertunity      1
Name: pupper, dtype: int64

In [55]:
twitter_archive_clean['puppo'].value_counts()

puppo           25
puppologize      1
puppologized     1
puppoccino       1
pupposes         1
puppos           1
pupporazzi       1
pupporting       1
puppon           1
Name: puppo, dtype: int64

In [56]:
# Create a dictionary of all the different iterations of dog type
dog_type = {'doggos': 'doggo',
           'doggles': 'doggo',
           'dogg': 'doggo',
           'doggy': 'doggo',
           'floof': 'floofer',
           'floofy': 'floofer',
           'floofs': 'floofer',
           'puppers': 'pupper',
           'puppertunity': 'pupper',
           'puppologize': 'puppo',
           'puppon': 'puppo',
           'pupposes': 'puppo',
           'puppoccino': 'puppo',
           'pupporazzi': 'puppo',
           'puppos': 'puppo',
           'pupporting': 'puppo',
           'puppologized': 'puppo'}

## Tidiness Issue 1

Twitter archive has a column for each dog type

## Clean

Concatenate the four dog type columns into one column and remove the original four columns

In [57]:
cols = ['doggo', 'floofer', 'pupper', 'puppo']
twitter_archive_clean['dog_stage'] = twitter_archive_clean[cols].apply(lambda x: ','.join(x.dropna()), axis=1)

In [58]:
twitter_archive_clean = twitter_archive_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

In [59]:
# Test

twitter_archive_clean.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
1532,690005060500217858,NaN,NaN,2016-01-21 02:56:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""I'm the only one that ever does anything in t...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/690005060...,10,10,None,
1144,727286334147182592,NaN,NaN,2016-05-02 23:59:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I swear to god if we get sent another Blue Mad...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/727286334...,9,10,None,
1791,677335745548390400,NaN,NaN,2015-12-17 03:53:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Downright inspiring 12/10 https://t.co/vSLtYBWHcQ,NaN,NaN,NaN,https://vine.co/v/hbLbH77Ar67,12,10,None,
927,754856583969079297,NaN,NaN,2016-07-18 01:53:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Winnie. She's not a fan of the fast mo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/754856583...,11,10,Winnie,
2322,666430724426358785,NaN,NaN,2015-11-17 01:40:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Oh boy what a pup! Sunglasses take this one to...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666430724...,6,10,None,
756,778650543019483137,NaN,NaN,2016-09-21 17:42:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Strider. He thinks he's a sorority girl. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778650543...,10,10,Strider,
874,761292947749015552,NaN,NaN,2016-08-04 20:09:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Bonaparte. He's pupset because it's cloud...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/761292947...,11,10,Bonaparte,
1519,690735892932222976,NaN,NaN,2016-01-23 03:20:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Peaches. She's a Dingleberry Zand...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/690735892...,13,10,Peaches,floof
777,776088319444877312,NaN,NaN,2016-09-14 16:00:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I can't tap the screen to make the hearts appe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/776088319...,10,10,None,
1060,741438259667034112,NaN,NaN,2016-06-11 01:13:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tucker. He's still figuring out couche...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/741438259...,9,10,Tucker,


In [60]:
twitter_archive_clean['dog_stage'].value_counts()

                 1766
pupper            224
doggo              74
puppo              24
puppers            23
floof              15
doggo,pupper       10
floofer             9
doggos              8
floofy              3
doggles             3
dogg                2
doggy               2
doggo,floofer       1
floofs              1
puppologize         1
puppologized        1
pupposes            1
puppoccino          1
puppertunity        1
puppos              1
pupporazzi          1
doggo,puppon        1
doggo,puppo         1
pupporting          1
Name: dog_stage, dtype: int64

## Clean

Change dog types back to the original type

In [61]:
def update_type(twitter_archive_clean):
    if twitter_archive_clean['dog_stage'] in dog_type.keys():
        update_dog = dog_type[twitter_archive_clean['dog_stage']]
        return update_dog
    else:
        return twitter_archive_clean['dog_stage']

twitter_archive_clean['dog_stage'] = twitter_archive_clean.apply(update_type, axis=1)

In [62]:
twitter_archive_clean['dog_stage'].value_counts()

                 1766
pupper            248
doggo              89
puppo              31
floofer            28
doggo,pupper       10
doggo,floofer       1
doggo,puppon        1
doggo,puppo         1
Name: dog_stage, dtype: int64

In [63]:
# puppon is not used as a dog type
twitter_archive_clean.dog_stage = twitter_archive_clean.dog_stage.replace('doggo,puppon', 'doggo')

In [64]:
twitter_archive_clean['dog_stage'].value_counts()

                 1766
pupper            248
doggo              90
puppo              31
floofer            28
doggo,pupper       10
doggo,floofer       1
doggo,puppo         1
Name: dog_stage, dtype: int64